# Load Packages

In [33]:
#Using smth means: pre-compile and load smth as a package.
using Pkg;
#Set the Gurobi path
ENV["GUROBI_HOME"] = "/Library/gurobi912/macos64/"
#For first timers (comment after): compile Gurobi with the given path
#Pkg.build("Gurobi")
#Add the packages that may not be installed.
Pkg.add(["Latexify", "Polyhedra", "Plots", "Makie", "CDDLib", "JuMP", "Gurobi", "AbstractPlotting", "JLD", "HDF5"])
#Load the packages
using JuMP, Latexify, Gurobi, LinearAlgebra, Polyhedra, CDDLib, Plots, SparseArrays, Makie, AbstractPlotting, JLD, HDF5

display("Initialization done! All set :-)")

   Resolving package versions...
  No Changes to `C:\Users\Alexr\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Alexr\.julia\environments\v1.6\Manifest.toml`


"Initialization done! All set :-)"

# Load Data

In [34]:
function get_data(instance)
    if (instance==1)
        data = load("../data/X1-3.jld")["X"]
    elseif(instance==2)
        data = load("../data/X2-2.jld")["X"]
    else
        data = load("../data/X3-2.jld")["X"]
    end
    return data
end

get_data (generic function with 1 method)

# Create Gurobi Problem

In [35]:
function get_new_model() 
    return Model(Gurobi.Optimizer)
end

get_new_model (generic function with 1 method)

# Calculate Distance Matrix

In [36]:
function get_dist_matrix(data)
    k = Array{Float64}(undef, size(data,1), size(data,1))
    for i = 1:size(data,1)
        for j = 1:size(data,1)
            k[i,j] = sqrt((data[i,1]-data[j,1])^2 + (data[i,2]-data[j,2])^2)
        end
    end
    return k
end

get_dist_matrix (generic function with 1 method)

# Create Constraints

In [37]:
function get_constraints(data, instance)
    if (instance == 1)
        n_min = 2
        n_max = 8
        d_max = 10
    else
        n_min = 4
        n_max = 6
        d_max = 9
    end    
    max_clients = Int(size(data,1))
    max_depots = Int(ceil(max_clients/n_min))
    return (max_clients, n_max, n_min, d_max, max_depots)
end

get_constraints (generic function with 1 method)

# Create y

In [38]:
function add_y(model, max_depots)
    return @variable(model, y[1:max_depots], base_name = "y", binary=true)
end

add_y (generic function with 1 method)

# Create x

In [39]:
function add_x(model, max_depots, max_clients)
    return @variable(model, x[1:max_clients,1:max_depots], base_name = "x", binary=true)
end

add_x (generic function with 1 method)

# Add objective

In [40]:
function add_objective(package)
    return @objective(model, Min, sum(y))
end

add_objective (generic function with 2 methods)

# Add Constraint 1
$$\sum_{c=1}^nx_{cd}\leq n_{max}*y_d\forall d$$

In [46]:
function add_constraint_1(model, max_clients, n_max, n_min, d_max, max_depots, k, x, y)
    for d in 1:max_depots
        constraint_expr = @expression(model, 0)
        for c in 1:max_clients
            constraint_expr += x[c,d]
        end
        @constraint(model, constraint_expr<=n_max*y[d])
    end
end

add_constraint_1 (generic function with 4 methods)

# Add constraint 2
$$\sum_{c=1}^nx_{cd}\geq n_{min}*y_d\forall d $$

In [42]:
function add_constraint_2(model, max_clients, n_max, n_min, d_max, max_depots, k, x, y)
    for d in 1:max_depots
        constraint_expr = @expression(model, 0)
        for c in 1:max_clients
            constraint_expr += x[c,d]
        end
        @constraint(model, constraint_expr>=n_min*y[d])
    end
end

add_constraint_2 (generic function with 3 methods)

# Constraint 3
$$\sum_{d=1}^nx_{cd}= 1\forall c$$

In [43]:
function add_constraint_3(model, max_clients, n_max, n_min, d_max, max_depots, k, x, y)
    for c in 1:max_clients
        constraint_expr = @expression(model, 0)
        for d in 1:max_depots
            constraint_expr += x[c,d]
        end
        @constraint(model, constraint_expr==1)
    end
end

add_constraint_3 (generic function with 3 methods)

# Constraint 4
$$\sum_{c_2=1}^nk_{{c_1}{c_2}}*x_{c_1d}*x_{c_2d}\leq d_{max}\forall c_1 \in C, \forall d \in D$$

In [44]:
function add_constraint_4(model, max_clients, n_max, n_min, d_max, max_depots, k, x, y)
    for d in 1:max_depots
        for c1 in 1:max_clients
            constraint_expr = @expression(model, 0)
            for c2 in 1:c1-1
                constraint_expr += k[c1,c2]*x[c1,d]*x[c2,d]
            end
            @constraint(model, constraint_expr<=d_max)
        end
    end
end

add_constraint_4 (generic function with 3 methods)

# Solve
## Instance 1

In [47]:
data_1 = get_data(1)
dist_matrix_1 = get_dist_matrix(data_1)
max_clients_1, n_max_1, n_min_1, d_max_1, max_depots_1 = get_constraints(data_1, 1)
model_1 = get_new_model()
x_1 = add_x(model_1, max_depots_1, max_clients_1)
y_1 = add_y(model_1, max_depots_1)
add_objective(model_1, y_1)
add_constraint_1(model_1, max_clients_1, n_max_1, n_min_1, d_max_1, max_depots_1, dist_matrix_1, x_1, y_1)
add_constraint_2(model_1, max_clients_1, n_max_1, n_min_1, d_max_1, max_depots_1, dist_matrix_1, x_1, y_1)
add_constraint_3(model_1, max_clients_1, n_max_1, n_min_1, d_max_1, max_depots_1, dist_matrix_1, x_1, y_1)
add_constraint_4(model_1, max_clients_1, n_max_1, n_min_1, d_max_1, max_depots_1, dist_matrix_1, x_1, y_1)
optimize!(model_1)

Academic license - for non-commercial use only


## Instance 2

In [ ]:
data_2 = get_data(2)
dist_matrix_2 = get_dist_matrix(data_2)
max_clients_2, n_max_2, n_min_2, d_max_2, max_depots_2 = get_constraints(data_2, 2)
model_2 = get_new_model()
x_2 = add_x(model_2, max_depots_2, max_clients_2)
y_2 = add_y(model_2, max_depots_2)
add_objective(model_2, y_2)
add_constraint_1(model_2, max_clients_2, n_max_2, n_min_2, d_max_2, max_depots_2, dist_matrix_2, x_2, y_2)
add_constraint_2(model_2, max_clients_2, n_max_2, n_min_2, d_max_2, max_depots_2, dist_matrix_2, x_2, y_2)
add_constraint_3(model_2, max_clients_2, n_max_2, n_min_2, d_max_2, max_depots_2, dist_matrix_2, x_2, y_2)
add_constraint_4(model_2, max_clients_2, n_max_2, n_min_2, d_max_2, max_depots_2, dist_matrix_2, x_2, y_2)
optimize!(model_2)

## Instance 3

In [ ]:
data_3 = get_data(3)
dist_matrix_3 = get_dist_matrix(data_3)
max_clients_3, n_max_3, n_min_3, d_max_3, max_depots_3 = get_constraints(data_3, 3)
model_3 = get_new_model()
x_3 = add_x(model_3, max_depots_3, max_clients_3)
y_3 = add_y(model_3, max_depots_3)
add_objective(model_3, y_3)
add_constraint_1(model_3, max_clients_3, n_max_3, n_min_3, d_max_3, max_depots_3, dist_matrix_3, x_3, y_3)
add_constraint_2(model_3, max_clients_3, n_max_3, n_min_3, d_max_3, max_depots_3, dist_matrix_3, x_3, y_3)
add_constraint_3(model_3, max_clients_3, n_max_3, n_min_3, d_max_3, max_depots_3, dist_matrix_3, x_3, y_3)
add_constraint_4(model_3, max_clients_3, n_max_3, n_min_3, d_max_3, max_depots_3, dist_matrix_3, x_3, y_3)
optimize!(model_3)